llm_zoomcamp module 1 
01-intro
import faq documents; run elastic search in docker; api call to openrouter.ai using key from .env file
model used for homework -- deepseek/deepseek-r1-0528:free

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9400:9400 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6

In [4]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [5]:
es_client.info()

ObjectApiResponse({'name': '4ec52bfb360d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Y3q-_qENRriM7PKiINvjaw', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm #adding progress_bar
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [8]:
query = "How do execute a command on a Kubernetes pod?"

In [9]:
# example
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], #, "section"
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [10]:
# cohort question 3
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], #, "section"
                    "type": "best_fields"
                }
            }
            }
        }
    }


In [11]:
response = es_client.search(index=index_name, body = search_query)

In [12]:
response

ObjectApiResponse({'took': 276, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'mLWfb5cBd4rbbBLn1CMQ', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'J7Wfb5cBd4rbbBLn2yTz', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kuber

In [13]:
result_docs = []    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
result_docs    

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t

In [14]:
# cohort question 4
query = "How do copy a file to a Docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], #, "section"
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body = search_query)
response

ObjectApiResponse({'took': 65, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 340, 'relation': 'eq'}, 'max_score': 73.38676, 'hits': [{'_index': 'course-questions', '_id': 'mLWfb5cBd4rbbBLn1CMQ', '_score': 73.38676, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 't7Wfb5cBd4rbbBLn1SPI', '_score': 66.688705, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo c

In [15]:
response

ObjectApiResponse({'took': 65, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 340, 'relation': 'eq'}, 'max_score': 73.38676, 'hits': [{'_index': 'course-questions', '_id': 'mLWfb5cBd4rbbBLn1CMQ', '_score': 73.38676, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 't7Wfb5cBd4rbbBLn1SPI', '_score': 66.688705, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo c

In [16]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [17]:
contexts = []
for hit in response['hits']['hits']:
    source = hit['_source']
    question = source.get('question', '')
    text = source.get('text', '')
    entry = context_template.format(question=question, text=text)
    contexts.append(entry)

context_output = '\n\n'.join(contexts)

print(context_output)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [19]:
context = context_template.format(question=query, text=response)
prompt = prompt_template.format(question=query, context=context_output).strip()

In [20]:
len(prompt)

1446

In [24]:
import sys
from pathlib import Path

current_dir = Path().resolve()
project_root = current_dir.parent

# Add project root to sys.path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import nbimporter
from api_call import chat_with_openrouter
print(sys.path)

['/home/ubuser/llm-zoomcamp', '/home/ubuser/spark/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip', '/home/ubuser/spark/spark-3.3.2-bin-hadoop3/python', '/home/ubuser/llm-zoomcamp/01-intro', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/ubuser/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/local/lib/python3.10/dist-packages/eflomal-0.1-py3.10-linux-x86_64.egg', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [25]:
result = chat_with_openrouter(prompt)

In [26]:
print(result['usage'])

{'prompt_tokens': 334, 'completion_tokens': 635, 'total_tokens': 969}


In [27]:
print(result['choices'][0]['message']['content'])

To copy a file to a Docker container, use the `docker cp` command. The syntax is:  
```bash
docker cp /path/to/local/file container_id:/path/in/container
```  
**Steps**:  
1. Locate the container ID with `docker ps` if you don’t know it.  
2. Run `docker cp` with the local file path and the target path inside the container (including the container ID and destination directory).  

**Example**:  
```bash
docker cp ./myfile.txt abc1234def56:/app/data/  
```  
*Note: This works for **running containers**. For copying files during image building, use `COPY` in a Dockerfile (e.g., `COPY file.txt /destination`).*  

(Answer derived directly from Hrithik Kumar Advani's FAQ entry in the provided context.)
